In [1]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter

import tkinter as tk
from tkinter import filedialog

import os

from scipy import signal
import scipy.integrate as integrate

# %matplotlib qt

In [3]:
n_core = 10
N = 512
print(list(linspace(1,N+1-N//n_core,n_core,dtype='int')))
print(list(linspace(N//n_core,N,n_core,dtype='int')))

n_dt = 5000

# integer          , parameter :: ia(ni)=[1,257,513,769]
# integer          , parameter :: ib(ni)=[256,512,768,1024]

[1, 52, 103, 154, 205, 257, 308, 359, 410, 462]
[51, 102, 153, 204, 255, 307, 358, 409, 460, 512]


In [4]:
%run functions
matplotlib.rcParams.update({'font.size': 25})

In [5]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol

wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [8]:
file_path = '/home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024' # 20220610
# file_path = '/home/adrien/RemoteFS/Rivendel/Simulations/20220617/N1024' # 20220610
# file_path = '/home/adrien/RemoteFS/Rivendel/Simulations/20220512'
# file_path = '/home/adrien/RemoteFS/Rivendel/Simulations/20220510'

all_subdir = []
all_Temp = []
all_trjN = []
all_trj1 = []
all_xva_3DHarmo = []
all_xva_start = []

# selection of all directories where
# there are data files
for i,j in enumerate(os.walk(file_path)):
    if 'Try' in j[0]:
        all_subdir.append(j[0])
    # selection of all data files
    # interesting me
    for k,l in enumerate(j[2]):
        if 'TrjN' in l:
            all_trjN.append(j[0]+'/'+l)
        if 'Trj1' in l:
            all_trj1.append(j[0]+'/'+l)
            #print(l)
        if 'SimuType0' and '.dat' in l:
            all_Temp.append(j[0]+'/'+l)
        if 'xva_3D' in l:
            all_xva_3DHarmo.append(j[0]+'/'+l)
        if 'xva_start' in l:
            all_xva_start.append(j[0]+'/'+l)
        #print(j[0],'\n')
all_subdir.sort()
all_Temp.sort()
all_xva_3DHarmo.sort()
all_xva_start.sort()
all_trjN.sort()
# for i in range(len(all_subdir)):
#     print(all_subdir[i],'\n',all_Temp[i],'\n',all_trjN[i],'\n')

In [36]:
for j,k in enumerate(all_subdir):
    str_N  = all_Temp[j].find('_N')
    cond_N = '1024' # all_Temp[j][str_N+3:str_N+7]
    str_v  = all_subdir[j].find('DC')
    cond   = all_Temp[j][str_v:str_v+9]
    print(cond)
    outfile = 'Time_and_temp_RFHEAT_N'+ str(cond_N) + '_' + str(cond)
    
    data = loadtxt(all_Temp[j])
    time_s = data[:,0]
    T_CM = data[:,1:4]
    T_aux = data[:,4:]
    T_aux_avg = mean(T_aux,axis=1)
    r2_v2 = load_xv2rlim(all_trjN[j])
    save_xva = load_xyz_init_bin_DP(all_xva_start[j].strip('.bin'))
    alpha = max(sqrt(save_xva[0]**2+save_xva[1]**2))/max(save_xva[2])
    #r2    = r2_v2[0:3,:]
    #v2    = r2_v2[3:6,:]
    #rlim  = r2_v2[6:9,:]
    
    np.savez( outfile, time=time_s, temp=T_aux_avg, r2_v2_rlim=r2_v2, alpha=alpha) # dirname_hc=re.sub(r'\\',r'/',dirname_hc)
    print(outfile) # , r2_v2_rlim=r2_v2

DC05_RF05
[36000000]
[36000000]
Time_and_temp_RFHEAT_N1024_DC05_RF05
DC06_RF06
[36000000]
[36000000]
Time_and_temp_RFHEAT_N1024_DC06_RF06
DC07_RF07
[36000000]
[36000000]
Time_and_temp_RFHEAT_N1024_DC07_RF07
DC08_RF08
[36000000]
[36000000]
Time_and_temp_RFHEAT_N1024_DC08_RF08
DC09_RF09
[36000000]
[36000000]
Time_and_temp_RFHEAT_N1024_DC09_RF09
DC10_RF10
[36000000]
[36000000]
Time_and_temp_RFHEAT_N1024_DC10_RF10
DC11_RF11
[36000000]
[36000000]
Time_and_temp_RFHEAT_N1024_DC11_RF11
DC12_RF12
[36000000]
[36000000]
Time_and_temp_RFHEAT_N1024_DC12_RF12
DC13_RF13
[36000000]
[36000000]
Time_and_temp_RFHEAT_N1024_DC13_RF13


In [15]:
all_Temp[9]

'/home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC19_RF19/Try00/Temp_SimuType0_N01024_Vrf0064_Udc0.1456D+02V_nt1000.dat'

In [23]:
for j in range(9,19):
#     j = all_subdir[k]
    str_N  = all_Temp[j].find('_N')
    cond_N = all_Temp[j][str_N+3:str_N+7]
    str_v  = all_subdir[j].find('DC')
    cond   = all_Temp[j][str_v:str_v+9]
    print(cond)
    outfile = 'Time_and_temp_RFHEAT_N'+ str(cond_N) + '_' + str(cond)

    data = loadtxt(all_Temp[j])
    time_s = data[:,0]
    T_CM = data[:,1:4]
    T_aux = data[:,4:]
    T_aux_avg = mean(T_aux,axis=1)
    r2_v2 = load_xv2rlim(all_trjN[j])
    save_xva = load_xyz_init_bin_DP(all_xva_start[j].strip('.bin'))
    alpha = max(sqrt(save_xva[0]**2+save_xva[1]**2))/max(save_xva[2])
    #r2    = r2_v2[0:3,:]
    #v2    = r2_v2[3:6,:]
    #rlim  = r2_v2[6:9,:]

    np.savez( outfile, time=time_s, temp=T_aux_avg, r2_v2_rlim=r2_v2, alpha=alpha) # dirname_hc=re.sub(r'\\',r'/',dirname_hc)
    print(outfile) # , r2_v2_rlim=r2_v2

DC19_RF19
[36000000]
[36000000]
Time_and_temp_RFHEAT_N1024_DC19_RF19
DC20_RF20
[36000000]
[36000000]
Time_and_temp_RFHEAT_N1024_DC20_RF20
DC21_RF21
[36000000]
[36000000]
Time_and_temp_RFHEAT_N1024_DC21_RF21
DC22_RF22
[36000000]
[36000000]
Time_and_temp_RFHEAT_N1024_DC22_RF22
DC23_RF23
[36000000]
[36000000]
Time_and_temp_RFHEAT_N1024_DC23_RF23
DC24_RF24
[36000000]
[36000000]
Time_and_temp_RFHEAT_N1024_DC24_RF24
DC25_RF25
[36000000]
[36000000]
Time_and_temp_RFHEAT_N1024_DC25_RF25
DC26_RF26
[36000000]
[36000000]
Time_and_temp_RFHEAT_N1024_DC26_RF26
DC27_RF27
[36000000]
[36000000]
Time_and_temp_RFHEAT_N1024_DC27_RF27
DC28_RF28
[36000000]
[36000000]
Time_and_temp_RFHEAT_N1024_DC28_RF28


In [21]:
print(all_subdir[12])
print(all_xva_start[12])

/home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC22_RF22/Try00
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC22_RF22/Try00/xva_start_SimuType0_N01024_Vrf0063_Udc0.1376D+02V_nt1000.bin
